In [ ]:
# Import all relevant libraries

import warnings
def fxn(): 
	warnings.warn("deprecated",DeprecationWarning)

with warnings.catch_warnings( ):
    warnings.simplefilter("ignore")
    fxn( )
    
import numpy as np
import h5py

In [ ]:
# Load data from specific trial
def get_trial(trial_num):    
    trial = h5py.File('../data/A0' + str(trial_num) + 'T_slice.mat', 'r')
    X = np.copy(trial['image'])
    y = np.copy(trial['type'])
    y = y[0,0:X.shape[0]:1]
    y = np.asarray(y, dtype=np.int32)
    y -= 769                            # shift class labels to [0-3]
    X = np.nan_to_num(X)[:, :22, :]     # remove EOG channels
    return X, y

''' 
for trial_num in xrange(1, 9):
    X, y = get_trial(trial_num)
'''

X, y = get_trial(1)

In [ ]:
print(X.shape)
print(y.shape)
print(y)

In [ ]:
from keras.models import Sequential
from keras.layers import Permute, Flatten, Softmax, Dense, Conv1D, Conv2D, Conv2DTranspose, AveragePooling2D, Activation, Reshape
from keras.backend import permute_dimensions

In [ ]:
model = Sequential()
model.add(Conv2D(filters=40, kernel_size=(22,1), activation='elu', input_shape=(1000, 22, 1), strides=1, data_format="channels_last"))
print(model.output_shape)
model.add(Permute((3, 2, 1)))
print(model.output_shape)
model.add(Conv2D(filters=40, kernel_size=(22, 1), activation='elu', data_format="channels_first"))
print(model.output_shape)
model.add(Permute((3, 2, 1)))
print(model.output_shape)
model.add(AveragePooling2D(pool_size=(75,1), strides=(15,1)))
print(model.output_shape)
model.add(Flatten())
model.add(Dense(units=4, kernel_initializer='glorot_normal'))
print(model.output_shape)

In [ ]:
from keras.optimizers import Adam
Adam = Adam(lr=0.15)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
one_hot = [[] for val in y]
for idx, val in enumerate(y):
    if val == 769:
        cur = [1, 0, 0, 0]
    elif val == 770:
        cur = [0, 1, 0, 0]
    elif val == 771:
        cur = [0, 0, 1, 0]
    else:
        cur = [0, 0, 0, 1]
    one_hot[idx] = cur
one_hot = np.asarray(one_hot)
print(one_hot.shape)
print(X.shape)
print(X.transpose(0, 2,1)[:, 0:22, :, None].shape)
print(one_hot[0:22].shape)




model.fit(X.transpose(0,2,1)[:, :, 0:22, None], y, epochs=10, batch_size=128)